# Intro
This is a simple data clean manipulation demonstration using a snippet of Census Bureau ethnicty data for New York City. There are Census tracts for each county.

In [1]:
import pandas as pd
import numpy as np

#read in csv file as a dataframe
df = pd.read_csv('demographics.csv', header = 0, names = ['geoid', 'geoidname', 'population', 'asian', 'black', 'hispanic', 'white'])

In [2]:
df.head()

,geoid,geoidname,population,asian,black,hispanic,white
0,36001000100,"Census Tract 1, Albany County, New York",2139,55,1343,198,466
1,36001000200,"Census Tract 2, Albany County, New York",6046,54,4455,648,699
2,36001000300,"Census Tract 3, Albany County, New York",5888,211,2077,615,2740
3,36001000401,"Census Tract 4.01, Albany County, New York",2227,56,198,53,1896
4,36001000403,"Census Tract 4.03, Albany County, New York",4415,443,407,201,3275


In [3]:
df.columns

Index(['geoid', 'geoidname', 'population', 'asian', 'black', 'hispanic',
       'white'],
      dtype='object')

In [4]:
df.dtypes

geoid          int64
geoidname     object
population    object
asian          int64
black          int64
hispanic       int64
white          int64
dtype: object

## Data Cleaning
First we try to transform columns ```geoidname``` and ```population``` into str and integers respectively. We are trying to extract only the county name from the ```geoidname``` column.
```
df['geoidname'] = df['geoidname'].astype(str)
df['population'] = df['population'].astype(int)
```

We receive the error ```invalid literal for int() with base 10: '4427(r46992)'``` when we attempt to transform the ```population column```. Python is having trouble with recognizing an object with parenthesis (indicating an revision of the population statistic) behind it. You can either edit every single row for every error
```
df['population'][673] = 4427
df['population'][696] = 5126
df['population'][725] = 6011
df['population'][736] = 0
df['population'][2840] = 2184
df['population'][2923] = 4967
df['population'][3107] = 3481
df['population'][3109] = 2450
df['population'][3110] = 2515
```
or apply a function that extracts everything in front of parenthesis '('. 

In [86]:
#First transform both to str explicitly as Python will throw an error.
df['geoidname'] = df['geoidname'].astype(str)
df['population'] = df['population'].astype(str)


### Note
It may be tempting to simply use ```return item[:item.find('(')]``` for our extraction function, but this will return an empty strings for single digits numbers or different numbers for bigger numbers as demonstrated below. This will also cause problems when you try to convert the column ```population``` to integers as it would not be able to covert null values.

In [9]:
a = '1'
print("Result:", a[:a.find('(')])
a = '10'
print("Result:", a[:a.find('(')])

Result: 
Result: 1


In [88]:
def get_county(item):
    if '(' in item:
        return item[:item.find('(')]
    else:
        return item

In [89]:
df['population'] = df['population'].apply(get_county)

In [90]:
#You can confirm the function has worked
df['population'][673]

'4427'

In [91]:
df.to_csv('look.txt')

In [92]:
#You will need to first convert to float and then integer
df['population'] = df['population'].astype(float).astype(int)
#or alternatively use
df['population'] = pd.to_numeric(df['population'])

In [93]:
def get_county_name(item):
    split_item = item.split(",")
    return split_item[1]
    
df['geoidname'] = df['geoidname'].apply(get_county_name)
df.head()

,geoid,geoidname,population,asian,black,hispanic,white
0,36001000100,Albany County,2139,55,1343,198,466
1,36001000200,Albany County,6046,54,4455,648,699
2,36001000300,Albany County,5888,211,2077,615,2740
3,36001000401,Albany County,2227,56,198,53,1896
4,36001000403,Albany County,4415,443,407,201,3275


## Adding new columns
Now suppose we wish add new columns to our dataframe that give us:

- each ethnity to total population ratio
- the number of tracts in each county (not counting those with population 0)

and 

- the number of tracts in each county that have majority (for example, the number of tracts in Albany that have more than 50% hispanics) for each ethnicity.

In [94]:
for column in ['asian', 'black','hispanic','white']:
    df[column] = pd.to_numeric(df[column])
    df[f'{column}_share'] = df[column] / df['population']

In [95]:
df.head()

,geoid,geoidname,population,asian,black,hispanic,white,asian_share,black_share,hispanic_share,white_share
0,36001000100,Albany County,2139,55,1343,198,466,0.025713,0.627863,0.092567,0.217859
1,36001000200,Albany County,6046,54,4455,648,699,0.008932,0.736851,0.107178,0.115614
2,36001000300,Albany County,5888,211,2077,615,2740,0.035836,0.352751,0.104450,0.465353
3,36001000401,Albany County,2227,56,198,53,1896,0.025146,0.088909,0.023799,0.851370
4,36001000403,Albany County,4415,443,407,201,3275,0.100340,0.092186,0.045527,0.741789


In [96]:
#Take note that some tracts just have no population
df.iloc[736]

geoid                  36027640001
geoidname          Dutchess County
population                       0
asian                            0
black                            0
hispanic                         0
white                            0
asian_share                    NaN
black_share                    NaN
hispanic_share                 NaN
white_share                    NaN
Name: 736, dtype: object

Now we try to count the number of tracts in each county. We need to filter out those with population 0. The first method that comes to mind is grouping. Here's a first try.

```df['tract_count'] = df[df['population']>0].groupby('geoidname')['geoidname'].transform(len)```

Unfortunately, while this is successful to the point where it subsets the dataframe where only those with positive population are filtered, groups it by county name, and returns the length of each group, which is precisely the number of positive population tracts in each county, this leaves the rows that have ```population = 0``` with a NaN value in the ```tract_count```column. 

Here's a better alternative.

In [98]:
#Creates a column with 1 and 0 (corresponding to the bool value)
df['tract_count'] = df['population'] > 0
df['tract_count'] = df.groupby('geoidname')['tract_count'].transform(sum)
df['tract_count'] = df['tract_count'].astype(int)

0       75
1       75
2       75
3       75
4       75
        ..
4914     5
4915     5
4916     5
4917     5
4918     5
Name: tract_count, Length: 4919, dtype: int32

In [100]:
#you can check that correct number of tracts have been put for bronx county
df[df['geoidname'] == ' Bronx County']['tract_count']

88     336.0
89     336.0
90     336.0
91     336.0
92     336.0
       ...  
422    336.0
423    336.0
424    336.0
425    336.0
426    336.0
Name: tract_count, Length: 339, dtype: float64

Now the final part majority tract count. We use the fact that if one ethnicty is more than 50% for a tract, no other ethnicity will be majority for a tract. We separately indicate if a tract has no majority ethnicity over 50% of a population.

In [103]:
#First, create columns indicating whether the share is more than 50%
for column in ['asian', 'black','hispanic', 'white']:
    df[f"{column}_majority_tracts"] = df[f'{column}_share'] > 0.5
df.head()

,geoid,geoidname,population,asian,black,hispanic,white,asian_share,black_share,hispanic_share,white_share,tract_count,asian_majority_tracts,black_majority_tracts,hispanic_majority_tracts,white_majority_tracts
0,36001000100,Albany County,2139,55,1343,198,466,0.025713,0.627863,0.092567,0.217859,75.0,False,True,False,False
1,36001000200,Albany County,6046,54,4455,648,699,0.008932,0.736851,0.107178,0.115614,75.0,False,True,False,False
2,36001000300,Albany County,5888,211,2077,615,2740,0.035836,0.352751,0.104450,0.465353,75.0,False,False,False,False
3,36001000401,Albany County,2227,56,198,53,1896,0.025146,0.088909,0.023799,0.851370,75.0,False,False,False,True
4,36001000403,Albany County,4415,443,407,201,3275,0.100340,0.092186,0.045527,0.741789,75.0,False,False,False,True


Now create a no majority tract column.

Try to create cases of 1 and 0 in the equation any majority tracts? $= (a + b + c + d - 1)*(-1)$ to figure out what's going on, where $a,b,c,d$ corresponds to the booleans values regarding any majority tracts for each respective ethnicity. 

In [104]:
df["no_majority_tracts"] = df["asian_majority_tracts"] + df["black_majority_tracts"] + df["hispanic_majority_tracts"] +df["white_majority_tracts"] - 1
df["no_majority_tracts"] = df["no_majority_tracts"]*(-1)

C:\Users\Jae-Hoon Kim\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]


In [105]:
df.head()

,geoid,geoidname,population,asian,black,hispanic,white,asian_share,black_share,hispanic_share,white_share,tract_count,asian_majority_tracts,black_majority_tracts,hispanic_majority_tracts,white_majority_tracts,no_majority_tracts
0,36001000100,Albany County,2139,55,1343,198,466,0.025713,0.627863,0.092567,0.217859,75.0,False,True,False,False,0
1,36001000200,Albany County,6046,54,4455,648,699,0.008932,0.736851,0.107178,0.115614,75.0,False,True,False,False,0
2,36001000300,Albany County,5888,211,2077,615,2740,0.035836,0.352751,0.104450,0.465353,75.0,False,False,False,False,1
3,36001000401,Albany County,2227,56,198,53,1896,0.025146,0.088909,0.023799,0.851370,75.0,False,False,False,True,0
4,36001000403,Albany County,4415,443,407,201,3275,0.100340,0.092186,0.045527,0.741789,75.0,False,False,False,True,0


In [107]:
#Now use the same trick we used for tract_count previously
for column in ['asian', 'black','hispanic', 'white', 'no']:
    df[f'{column}_majority_tracts'] = df.groupby('geoidname')[f'{column}_majority_tracts'].transform(sum)


In [108]:
df.head()

,geoid,geoidname,population,asian,black,hispanic,white,asian_share,black_share,hispanic_share,white_share,tract_count,asian_majority_tracts,black_majority_tracts,hispanic_majority_tracts,white_majority_tracts,no_majority_tracts
0,36001000100,Albany County,2139,55,1343,198,466,0.025713,0.627863,0.092567,0.217859,75.0,0.0,6.0,0.0,63.0,6
1,36001000200,Albany County,6046,54,4455,648,699,0.008932,0.736851,0.107178,0.115614,75.0,0.0,6.0,0.0,63.0,6
2,36001000300,Albany County,5888,211,2077,615,2740,0.035836,0.352751,0.104450,0.465353,75.0,0.0,6.0,0.0,63.0,6
3,36001000401,Albany County,2227,56,198,53,1896,0.025146,0.088909,0.023799,0.851370,75.0,0.0,6.0,0.0,63.0,6
4,36001000403,Albany County,4415,443,407,201,3275,0.100340,0.092186,0.045527,0.741789,75.0,0.0,6.0,0.0,63.0,6


In [109]:
df.head(20)

,geoid,geoidname,population,asian,black,hispanic,white,asian_share,black_share,hispanic_share,white_share,tract_count,asian_majority_tracts,black_majority_tracts,hispanic_majority_tracts,white_majority_tracts,no_majority_tracts
0,36001000100,Albany County,2139,55,1343,198,466,0.025713,0.627863,0.092567,0.217859,75.0,0.0,6.0,0.0,63.0,6
1,36001000200,Albany County,6046,54,4455,648,699,0.008932,0.736851,0.107178,0.115614,75.0,0.0,6.0,0.0,63.0,6
2,36001000300,Albany County,5888,211,2077,615,2740,0.035836,0.352751,0.104450,0.465353,75.0,0.0,6.0,0.0,63.0,6
3,36001000401,Albany County,2227,56,198,53,1896,0.025146,0.088909,0.023799,0.851370,75.0,0.0,6.0,0.0,63.0,6
4,36001000403,Albany County,4415,443,407,201,3275,0.100340,0.092186,0.045527,0.741789,75.0,0.0,6.0,0.0,63.0,6
5,36001000404,Albany County,5203,371,707,647,3327,0.071305,0.135883,0.124351,0.639439,75.0,0.0,6.0,0.0,63.0,6
6,36001000501,Albany County,3412,147,1058,377,1658,0.043083,0.310082,0.110492,0.485932,75.0,0.0,6.0,0.0,63.0,6
7,36001000502,Albany County,3448,221,434,276,2419,0.064095,0.125870,0.080046,0.701566,75.0,0.0,6.0,0.0,63.0,6
8,36001000600,Albany County,3676,310,1271,419,1465,0.084331,0.345756,0.113983,0.398531,75.0,0.0,6.0,0.0,63.0,6
9,36001000700,Albany County,3821,23,2889,383,360,0.006019,0.756085,0.100236,0.094216,75.0,0.0,6.0,0.0,63.0,6
